# Data Preparation

## Import and Settings

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import dask
import dask.dataframe as dd
import dask.array as da

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

In [6]:
df = dd.read_csv('prepared_ds.csv')

## Jonas
- Naive Bayes
- logistische Regression

## Ferdi
- Stochastic Gradient Descent
- Nearest Neighbors

## Basti
- Decision Trees
  - Ensemble Methods (GradientBoostingClassifier, Random, AdaBoost)
- Support Vector Machines

## Ron
- Neuronale Netze
- Gaussian Process (GPC)

## df_fast initialisieren (size = 0.01)

## train, test split

## trainieren (80, 20, random_state = 1)

In [8]:
train, test, train_lbl, test_lbl = train_test_split(X, Y, test_size=0.2, random_state=1)

NameError: name 'X' is not defined

### M1

### ... Model 2

## Accuracy

## confusion matrix (fp (wenn 1 (positiv) = DDoS) minimieren)

## Modell speichern